In [18]:
import random, numpy as np
alphap = 2
expon = 2 * alphap - 1
probp = 0.75

In [19]:
def betadist(alpha):
        return random.betavariate(alpha,alpha)

In [20]:
def decision(probability):
    """
    decides with a given probability whether to keep the right part
    """
    if float(probability) > random.random():
        return True
    else: 
        return False

In [21]:
def splitting(left,right):
    """
    splits a given segment. left and right are endpoints of the segment
    """
    segment = right - left
    xL = segment * betadist(alphap)
    xR = segment - xL
    splitpoint = left + xL
    flag = decision(probp)
    xLp = xL**expon
    xRp = xR**expon
    change = xLp + xRp - segment**expon
    return splitpoint, flag, xLp, xRp, change

In [22]:
def fraclen(List, index):
    sum = 0
    for i in range(len(List[:index+1])):
        sum += List[i]
    return sum

In [23]:
def pickindex(expons, exponsum):
    """picks up a segment to be subsequently split"""
    r = random.uniform(0, exponsum)
    for index in range(len(expons)):
            if fraclen(expons, index) < r:
                continue
            else:
                return index

In [34]:
points = [0.,1.]
flags = [True]
expons = [1.]
exponsum = 1.0

for i in range(5):
    index = pickindex(expons, exponsum)
    if flags[index] == True:
        left = points[index]
        right = points[index+1]
        splitpoint, flag, xLp, xRp, change = splitting(left,right)
        points.insert(index+1,splitpoint)
        flags.insert(index+1,flag)
        del expons[index]
        expons.insert(index,xLp)
        expons.insert(index+1,xRp)
        exponsum += change

print(points)
print(flags)
print(expons)
print(exponsum)

[0.0, 0.16263201422551468, 0.27124249720340016, 0.4008055979544067, 0.6522969238077417, 0.8092225332897878, 1.0]
[True, True, False, True, True, True]
[0.004301482125259126, 0.0012811949992698206, 0.002174923568252256, 0.015906294953888383, 0.00386439465044724, 0.006943544653715941]
0.03447183495083273
